In [9]:
import pymongo
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import imageio
import os

# MongoDB connection URI and Database
uri = "mongodb://127.0.0.1:27017"
db_name = "tp_KBO"
collection_name = "enterprises"
gif_output_path = "regression_animation1.gif"  # Path to save the GIF
temp_image_folder = "temp_images"  # Temporary folder to store images

# Connect to MongoDB and fetch data
def fetch_data():
    print("Connexion à MongoDB...")
    client = pymongo.MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    
    projection = {
        "entity_number": 1,
        "status": 1,
        "start_date": 1,
        "activities.nace_description": 1,
        "establishments.activities.nace_description": 1,  # Include activities from establishments
        "_id": 0
    }
    
    print("Extraction des données de MongoDB...")
    data = list(collection.find({}, projection))
    print(f"Nombre d'entrées récupérées: {len(data)}")
    client.close()
    print("Connexion à MongoDB fermée.")
    return data

# Prepare the data for analysis
def prepare_data(data):
    print("Préparation des données pour l'analyse...")
    df = pd.DataFrame(data)
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['year'] = df['start_date'].dt.year
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype(int)

    def count_activities(activities, establishments):
        activity_count = len(activities) if isinstance(activities, list) else 0
        if isinstance(establishments, list):
            for est in establishments:
                activity_count += len(est.get('activities', []))
        return activity_count

    df['activity_count'] = df.apply(lambda row: count_activities(row.get('activities', []), row.get('establishments', [])), axis=1)
    print(f"Données préparées avec succès. Nombre d'entrées valides: {len(df)}")
    return df

# Perform linear regression by year
def perform_regression(df):
    print("Réalisation de la régression linéaire progressive par année...")
    # List to store images for the GIF
    images = []
    
    # Create folder if it doesn't exist
    if not os.path.exists(temp_image_folder):
        os.makedirs(temp_image_folder)

    # Sort data by year and iterate year by year
    df_sorted = df.sort_values(by='year')
    unique_years = df_sorted['year'].unique()

    for i, year in enumerate(unique_years, start=1):
        # Filter data up to the current year
        df_current = df_sorted[df_sorted['year'] <= year]
        X = df_current['year'].values.reshape(-1, 1)
        y = df_current['activity_count'].values
        
        # Create linear regression model
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)

        # Generate image for the current year
        image_path = os.path.join(temp_image_folder, f"frame_{year}.png")
        plot_progress(X, y, y_pred, model.coef_[0], model.intercept_, year, image_path)
        images.append(image_path)

    print(f"Régression terminée après {len(unique_years)} années.")
    return images

# Plot the results and save each frame as an image
def plot_progress(X, y, y_pred, slope, intercept, year, image_path):
    plt.figure(figsize=(10, 6))
    plt.scatter(X, y, color="blue", label="Données réelles")
    plt.plot(X, y_pred, color="red", linewidth=2, label=f"Régression linéaire: y = {slope:.2f}x + {intercept:.2f}")
    
    plt.title(f"Régression du nombre d'activités jusqu'à l'année {year}")
    plt.xlabel("Année")
    plt.ylabel("Nombre d'activités")
    plt.legend()
    plt.grid(True)
    plt.savefig(image_path)
    plt.close()
    print(f"Image sauvegardée pour l'année {year}: {image_path}")

# Combine all the images into a GIF
def create_gif(image_paths, gif_output_path):
    print("Création du GIF...")
    images = [imageio.imread(image_path) for image_path in image_paths]
    imageio.mimsave(gif_output_path, images, duration=0.5)
    print(f"GIF sauvegardé sous {gif_output_path}")

# Clean up temporary images after GIF creation
def cleanup_images(image_paths):
    print("Nettoyage des images temporaires...")
    for image_path in image_paths:
        os.remove(image_path)
    if os.path.exists(temp_image_folder):
        os.rmdir(temp_image_folder)
    print("Images temporaires supprimées.")

# Main function to execute the workflow
def main():
    print("Début du processus...")
    
    data = fetch_data()
    df = prepare_data(data)
    
    # Perform regression and generate images
    image_paths = perform_regression(df)
    
    # Create GIF from images
    create_gif(image_paths, gif_output_path)
    
    # Cleanup temporary images
    cleanup_images(image_paths)
    
    print("Processus terminé.")

if __name__ == "__main__":
    main()


Début du processus...
Connexion à MongoDB...
Extraction des données de MongoDB...


KeyboardInterrupt: 

In [3]:
import pymongo
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import imageio
import os

# MongoDB connection URI and Database
uri = "mongodb://127.0.0.1:27017"
db_name = "tp_KBO"
collection_name = "enterprises"

# Directory for saving frames
frames_dir = "frames/"
gif_filename = "regression_animation.gif"

# Connect to MongoDB and fetch data
def fetch_data():
    print("Connexion à MongoDB...")
    client = pymongo.MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    
    # Projection to retrieve only relevant fields
    projection = {
        "entity_number": 1,
        "status": 1,
        "start_date": 1,
        "activities.nace_description": 1,
        "_id": 0
    }
    
    print("Extraction des données de MongoDB...")
    data = list(collection.find({}, projection))
    print(f"Nombre d'entrées récupérées: {len(data)}")
    client.close()
    print("Connexion à MongoDB fermée.")
    return data

# Prepare the data for analysis
def prepare_data(data):
    print("Préparation des données pour l'analyse...")
    df = pd.DataFrame(data)

    # Convert start_date to datetime and extract the year
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['year'] = df['start_date'].dt.year

    # Filter rows where year is not NaN
    df = df.dropna(subset=['year'])
    
    # Convert year to numeric values (independent variable)
    df['year'] = df['year'].astype(int)

    # Calculate the number of activities per enterprise per year
    df['activity_count'] = df['activities'].apply(lambda x: len(x) if isinstance(x, list) else 0)

    # Group by year to calculate total number of activities per year
    df_grouped = df.groupby('year')['activity_count'].sum().reset_index()

    print(f"Données préparées avec succès. Nombre d'années : {len(df_grouped)}")
    return df_grouped

# Perform linear regression
def perform_regression(df):
    print("Réalisation de la régression linéaire...")
    X = df['year'].values.reshape(-1, 1)  # Independent variable
    y = df['activity_count'].values  # Dependent variable
    
    model = LinearRegression()
    model.fit(X, y)
    
    y_pred = model.predict(X)  # Predictions
    
    slope = model.coef_[0]  # Slope
    intercept = model.intercept_  # Intercept
    
    # Calculate R² and RMSE
    r_squared = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    
    print(f"Régression terminée. Pente: {slope:.2f}, Ordonnée à l'origine: {intercept:.2f}")
    print(f"Coefficient de détermination (R²): {r_squared:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    
    return X, y, y_pred, slope, intercept, r_squared, rmse

# Save each frame of the regression as an image
def save_frames(X, y, y_pred, slope, intercept, r_squared, rmse):
    print("Création des frames pour le GIF...")
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    # Complete line for regression (based on the entire dataset)
    y_pred_complete = slope * X + intercept

    for i in range(1, len(X) + 1):
        plt.figure(figsize=(10, 6))
        
        # Plot all the real data points
        plt.scatter(X, y, color="blue", label="Données réelles")

        # Plot the partial regression line
        plt.plot(X[:i], y_pred[:i], color="red", linewidth=2, label=f"Régression partielle: y = {slope:.2f}x + {intercept:.2f}")
        
        # Plot the full regression line
        plt.plot(X, y_pred_complete, color="green", linestyle="--", label="Ligne complète de régression")

        plt.title("Régression du nombre total d'activités par année")
        plt.xlabel("Année")
        plt.ylabel("Nombre total d'activités")
        plt.legend()
        plt.grid(True)
        
        # Show R² and RMSE on each frame
        plt.text(0.05, 0.95, f"R²: {r_squared:.2f}\nRMSE: {rmse:.2f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
        
        # Save each frame as an image
        frame_path = os.path.join(frames_dir, f"frame_{i:03d}.png")
        plt.savefig(frame_path)
        plt.close()
    
    print(f"Frames sauvegardées dans le dossier '{frames_dir}'.")

# Create a GIF from the saved frames
def create_gif():
    print("Création du GIF...")
    images = []
    for file_name in sorted(os.listdir(frames_dir)):
        if file_name.endswith(".png"):
            file_path = os.path.join(frames_dir, file_name)
            images.append(imageio.imread(file_path))
    
    imageio.mimsave(gif_filename, images, fps=2)
    print(f"GIF créé avec succès: {gif_filename}")

# Main function to execute the workflow
def main():
    print("Début du processus...")
    
    # Fetch data from MongoDB
    data = fetch_data()
    
    # Prepare data for regression
    df_grouped = prepare_data(data)
    
    # Perform regression
    X, y, y_pred, slope, intercept, r_squared, rmse = perform_regression(df_grouped)
    
    # Save frames for the GIF
    save_frames(X, y, y_pred, slope, intercept, r_squared, rmse)
    
    # Create GIF from saved frames
    create_gif()
    
    print("Processus terminé.")

if __name__ == "__main__":
    main()


Début du processus...
Connexion à MongoDB...
Extraction des données de MongoDB...
Nombre d'entrées récupérées: 1890258
Connexion à MongoDB fermée.
Préparation des données pour l'analyse...
Données préparées avec succès. Nombre d'années : 175
Réalisation de la régression linéaire...
Régression terminée. Pente: 295.47, Ordonnée à l'origine: -559526.83
Coefficient de détermination (R²): 0.34
Root Mean Squared Error (RMSE): 22973.27
Création des frames pour le GIF...
Frames sauvegardées dans le dossier 'frames/'.
Création du GIF...


C:\Users\atdf2\AppData\Local\Temp\ipykernel_18632\957366286.py:135: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


GIF créé avec succès: regression_animation.gif
Processus terminé.


In [1]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import imageio

# MongoDB connection URI and Database
uri = "mongodb://127.0.0.1:27017"
db_name = "tp_KBO"
collection_name = "enterprises"

# Dossier pour enregistrer les images temporaires
frames_dir = "frames"
gif_filename = "activity_group_trends.gif"

# Connexion à MongoDB et extraction des données
def fetch_data():
    print("Connexion à MongoDB...")
    client = pymongo.MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]

    # Projeter uniquement les champs pertinents
    projection = {
        "entity_number": 1,
        "start_date": 1,
        "activities.activity_group": 1,  # Groupes d'activités
        "_id": 0
    }
    
    print("Extraction des données de MongoDB...")
    data = list(collection.find({}, projection))
    client.close()
    print(f"Nombre d'entrées récupérées: {len(data)}")
    
    return pd.DataFrame(data)

# Préparation des données pour l'analyse
def prepare_data(df):
    print("Préparation des données pour l'analyse...")
    
    # Convertir start_date en datetime et extraire l'année
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['year'] = df['start_date'].dt.year
    
    # Filtrer les lignes sans date valide
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype(int)
    
    # Exploser les listes de `activity_group` pour avoir une ligne par groupe d'activité
    df = df.explode('activities')
    
    # Filtrer uniquement les entrées qui ont un groupe d'activités valide
    df = df[df['activities'].notnull()]
    
    # Extraire le groupe d'activités
    df['activity_group'] = df['activities'].apply(lambda x: x.get('activity_group') if isinstance(x, dict) else None)
    
    # Filtrer les lignes sans `activity_group`
    df = df.dropna(subset=['activity_group'])
    
    print(f"Nombre d'entrées après préparation: {len(df)}")
    return df

# Compter les occurrences des groupes d'activités par année
def analyze_activity_group_trends(df):
    print("Analyse des tendances des groupes d'activités...")
    
    # Compter le nombre de fois que chaque `activity_group` apparaît pour chaque année
    activity_trend = df.groupby(['year', 'activity_group']).size().reset_index(name='count')
    
    # Filtrer pour garder les 10 groupes d'activités les plus fréquents
    top_activity_groups = activity_trend.groupby('activity_group')['count'].sum().nlargest(10).index
    top_activity_trend = activity_trend[activity_trend['activity_group'].isin(top_activity_groups)]
    
    return top_activity_trend

# Créer et enregistrer un graphique pour chaque année
def save_frames_by_year(activity_trend):
    print("Création des frames pour chaque année...")
    
    # Créer un dossier pour enregistrer les frames
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    # Générer un graphique pour chaque année de manière cumulative et enregistrer les images
    years = sorted(activity_trend['year'].unique())
    for year in years:
        plt.figure(figsize=(12, 8))
        
        # Filtrer les données cumulatives jusqu'à l'année courante
        cumulative_data = activity_trend[activity_trend['year'] <= year]
        
        sns.lineplot(data=cumulative_data, x='year', y='count', hue='activity_group', marker='o')

        plt.title(f"Tendances des groupes d'activités jusqu'à {year}")
        plt.xlabel("Année")
        plt.ylabel("Nombre d'occurrences du groupe d'activité")
        plt.legend(title="Groupe d'activités", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        # Enregistrer l'image pour cette année
        frame_filename = os.path.join(frames_dir, f"frame_{year}.png")
        plt.savefig(frame_filename)
        plt.close()

    print(f"Frames sauvegardées dans le dossier '{frames_dir}'.")

# Créer un GIF à partir des frames enregistrées
def create_gif():
    print("Création du GIF...")
    images = []
    for file_name in sorted(os.listdir(frames_dir)):
        if file_name.endswith(".png"):
            file_path = os.path.join(frames_dir, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(gif_filename, images, fps=2)
    print(f"GIF créé avec succès: {gif_filename}")

# Supprimer les frames temporaires
def cleanup_frames():
    print("Nettoyage des frames temporaires...")
    for file_name in os.listdir(frames_dir):
        file_path = os.path.join(frames_dir, file_name)
        os.remove(file_path)
    os.rmdir(frames_dir)
    print("Frames temporaires supprimées.")

def main():
    print("Début du processus d'analyse des tendances des groupes d'activités...")

    # Étape 1 : Récupérer les données de MongoDB
    data = fetch_data()

    # Étape 2 : Préparer les données
    df_prepared = prepare_data(data)

    # Étape 3 : Analyser les tendances des groupes d'activités
    activity_trend = analyze_activity_group_trends(df_prepared)

    # Étape 4 : Sauvegarder les frames par année
    save_frames_by_year(activity_trend)

    # Étape 5 : Créer le GIF
    create_gif()

    # Étape 6 : Nettoyer les frames temporaires
    cleanup_frames()

    print("Processus terminé.")

if __name__ == "__main__":
    main()


Début du processus d'analyse des tendances des groupes d'activités...
Connexion à MongoDB...
Extraction des données de MongoDB...
Nombre d'entrées récupérées: 1890258
Préparation des données pour l'analyse...


C:\Users\atdf2\AppData\Local\Temp\ipykernel_22616\546181240.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].astype(int)


Nombre d'entrées après préparation: 2097252
Analyse des tendances des groupes d'activités...
Création des frames pour chaque année...
Frames sauvegardées dans le dossier 'frames'.
Création du GIF...


C:\Users\atdf2\AppData\Local\Temp\ipykernel_22616\546181240.py:118: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


GIF créé avec succès: activity_group_trends.gif
Nettoyage des frames temporaires...
Frames temporaires supprimées.
Processus terminé.


In [14]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import imageio

# MongoDB connection URI and Database
uri = "mongodb://127.0.0.1:27017"
db_name = "tp_KBO"
collection_name = "enterprises"

# Dossier pour enregistrer les images temporaires
frames_dir = "frames_postal_bar"
gif_filename = "activity_trends_bar_by_postal_code.gif"

# Connexion à MongoDB et extraction des données
def fetch_data():
    print("Connexion à MongoDB...")
    client = pymongo.MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]

    # Projeter uniquement les champs pertinents des établissements
    projection = {
        "entity_number": 1,
        "establishments.start_date": 1,  # Date de démarrage des établissements
        "establishments.addresses.zipcode": 1,  # Code postal des établissements
        "establishments.activities.activity_group": 1,  # Groupes d'activités des établissements
        "_id": 0
    }
    
    print("Extraction des données de MongoDB...")
    data = list(collection.find({}, projection))
    client.close()
    print(f"Nombre d'entrées récupérées: {len(data)}")
    
    return pd.DataFrame(data)

# Préparation des données pour l'analyse
def prepare_data(df):
    print("Préparation des données pour l'analyse...")
    
    # Exploser les listes d'établissements pour avoir une ligne par établissement
    df = df.explode('establishments')

    # Convertir establishments.start_date en datetime et extraire l'année
    df['establishment_start_date'] = df['establishments'].apply(lambda x: x.get('start_date') if isinstance(x, dict) else None)
    df['establishment_start_date'] = pd.to_datetime(df['establishment_start_date'], errors='coerce')
    df['year'] = df['establishment_start_date'].dt.year
    
    # Filtrer les lignes sans date valide
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype(int)

    # Extraire les activités et les adresses des établissements en vérifiant si les listes ne sont pas vides
    df['activity_group'] = df['establishments'].apply(
        lambda x: x.get('activities', [{}])[0].get('activity_group') if isinstance(x, dict) and len(x.get('activities', [])) > 0 else None
    )
    df['zipcode'] = df['establishments'].apply(
        lambda x: x.get('addresses', [{}])[0].get('zipcode') if isinstance(x, dict) and len(x.get('addresses', [])) > 0 else None
    )
    
    # Filtrer les lignes sans `activity_group` ou `zipcode`
    df = df.dropna(subset=['activity_group', 'zipcode'])
    
    print(f"Nombre d'entrées après préparation: {len(df)}")
    return df

# Regrouper les années en périodes de 50 ans
def group_years_by_period(df):
    print("Regroupement des années par périodes de 50 ans...")
    
    df['period'] = (df['year'] // 50) * 50  # Créer des périodes de 50 ans
    return df

# Compter les occurrences des groupes d'activités par code postal et période
def analyze_activity_trends_by_postal(df):
    print("Analyse des tendances des activités par code postal...")

    # Compter le nombre d'occurrences de chaque `activity_group` par code postal et par période
    activity_trend = df.groupby(['period', 'zipcode', 'activity_group']).size().reset_index(name='count')

    # Filtrer pour garder les 10 codes postaux les plus actifs
    top_postal_codes = activity_trend.groupby('zipcode')['count'].sum().nlargest(10).index

    # Regrouper les autres codes postaux sous 'Autres'
    activity_trend['zipcode_group'] = activity_trend['zipcode'].apply(lambda x: x if x in top_postal_codes else 'Autres')

    return activity_trend

# Créer et enregistrer un graphique à barres pour chaque période
def save_frames_by_period(activity_trend):
    print("Création des frames pour chaque période...")

    # Créer un dossier pour enregistrer les frames
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    # Générer un diagramme à barres pour chaque période et enregistrer les images
    periods = sorted(activity_trend['period'].unique())
    for period in periods:
        plt.figure(figsize=(12, 8))
        
        # Filtrer les données pour la période courante
        period_data = activity_trend[activity_trend['period'] == period]

        sns.barplot(data=period_data, x='zipcode_group', y='count', hue='activity_group', dodge=False, palette="muted")
        
        plt.title(f"Tendances des activités par code postal pour la période {period} - {period + 49}")
        plt.xlabel("Code Postal")
        plt.ylabel("Nombre d'occurrences d'activités")
        plt.legend(title="Groupe d'activités", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        # Enregistrer l'image pour cette période
        frame_filename = os.path.join(frames_dir, f"frame_{period}.png")
        plt.savefig(frame_filename)
        plt.close()

    print(f"Frames sauvegardées dans le dossier '{frames_dir}'.")

# Créer un GIF à partir des frames enregistrées
def create_gif():
    print("Création du GIF...")
    images = []
    for file_name in sorted(os.listdir(frames_dir)):
        if file_name.endswith(".png"):
            file_path = os.path.join(frames_dir, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(gif_filename, images, fps=0.3)
    print(f"GIF créé avec succès: {gif_filename}")

# Supprimer les frames temporaires
def cleanup_frames():
    print("Nettoyage des frames temporaires...")
    for file_name in os.listdir(frames_dir):
        file_path = os.path.join(frames_dir, file_name)
        os.remove(file_path)
    os.rmdir(frames_dir)
    print("Frames temporaires supprimées.")

def main():
    print("Début du processus d'analyse des tendances des activités par code postal...")

    # Étape 1 : Récupérer les données de MongoDB
    data = fetch_data()

    # Étape 2 : Préparer les données
    df_prepared = prepare_data(data)

    # Étape 3 : Regrouper les années en périodes de 50 ans
    df_grouped = group_years_by_period(df_prepared)

    # Étape 4 : Analyser les tendances des activités par code postal
    activity_trend = analyze_activity_trends_by_postal(df_grouped)

    # Étape 5 : Sauvegarder les frames par période
    save_frames_by_period(activity_trend)

    # Étape 6 : Créer le GIF
    create_gif()

    # Étape 7 : Nettoyer les frames temporaires
    cleanup_frames()

    print("Processus terminé.")

if __name__ == "__main__":
    main()


Début du processus d'analyse des tendances des activités par code postal...
Connexion à MongoDB...
Extraction des données de MongoDB...
Nombre d'entrées récupérées: 1890258
Préparation des données pour l'analyse...
Nombre d'entrées après préparation: 969879
Regroupement des années par périodes de 50 ans...
Analyse des tendances des activités par code postal...
Création des frames pour chaque période...
Frames sauvegardées dans le dossier 'frames_postal_bar'.
Création du GIF...


C:\Users\atdf2\AppData\Local\Temp\ipykernel_22060\3873807023.py:131: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


GIF créé avec succès: activity_trends_bar_by_postal_code.gif
Nettoyage des frames temporaires...
Frames temporaires supprimées.
Processus terminé.


In [5]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import imageio

# MongoDB connection URI and Database
uri = "mongodb://127.0.0.1:27017"
db_name = "tp_KBO"
collection_name = "enterprises"

# Dossier pour enregistrer les images temporaires
frames_dir = "frames_postal"
gif_filename = "activity_trends_by_postal_code.gif"

# Connexion à MongoDB et extraction des données
def fetch_data():
    print("Connexion à MongoDB...")
    client = pymongo.MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]

    # Projeter uniquement les champs pertinents des établissements, y compris leur date de démarrage
    projection = {
        "entity_number": 1,
        "establishments.start_date": 1,  # Date de démarrage des établissements
        "establishments.addresses.zipcode": 1,  # Code postal des établissements
        "establishments.activities.activity_group": 1,  # Groupes d'activités des établissements
        "_id": 0
    }
    
    print("Extraction des données de MongoDB...")
    data = list(collection.find({}, projection))
    client.close()
    print(f"Nombre d'entrées récupérées: {len(data)}")
    
    return pd.DataFrame(data)

# Préparation des données pour l'analyse
def prepare_data(df):
    print("Préparation des données pour l'analyse...")
    
    # Exploser les listes d'établissements pour avoir une ligne par activité et par adresse
    df = df.explode('establishments')

    # Convertir establishments.start_date en datetime et extraire l'année
    df['establishment_start_date'] = df['establishments'].apply(lambda x: x.get('start_date') if isinstance(x, dict) else None)
    df['establishment_start_date'] = pd.to_datetime(df['establishment_start_date'], errors='coerce')
    df['year'] = df['establishment_start_date'].dt.year
    
    # Filtrer les lignes sans date valide
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype(int)

    # Extraire les activités et les adresses des établissements en vérifiant si les listes ne sont pas vides
    df['activity_group'] = df['establishments'].apply(
        lambda x: x.get('activities', [{}])[0].get('activity_group') if isinstance(x, dict) and len(x.get('activities', [])) > 0 else None
    )
    df['zipcode'] = df['establishments'].apply(
        lambda x: x.get('addresses', [{}])[0].get('zipcode') if isinstance(x, dict) and len(x.get('addresses', [])) > 0 else None
    )
    
    # Filtrer les lignes sans `activity_group` ou `zipcode`
    df = df.dropna(subset=['activity_group', 'zipcode'])
    
    print(f"Nombre d'entrées après préparation: {len(df)}")
    return df

# Compter les occurrences des groupes d'activités par code postal et année
def analyze_activity_trends_by_postal(df):
    print("Analyse des tendances des activités par code postal...")
    
    # Compter le nombre d'occurrences de chaque `activity_group` par code postal et par année
    activity_trend = df.groupby(['year', 'zipcode', 'activity_group']).size().reset_index(name='count')
    
    # Filtrer pour garder les 10 codes postaux les plus actifs
    top_postal_codes = activity_trend.groupby('zipcode')['count'].sum().nlargest(10).index
    activity_trend['zipcode_group'] = activity_trend['zipcode'].apply(lambda x: x if x in top_postal_codes else 'Autres')
    
    return activity_trend

# Créer et enregistrer un graphique pour chaque année
def save_frames_by_year(activity_trend):
    print("Création des frames pour chaque année...")
    
    # Créer un dossier pour enregistrer les frames
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    # Générer un graphique pour chaque année de manière cumulative et enregistrer les images
    years = sorted(activity_trend['year'].unique())
    for year in years:
        plt.figure(figsize=(14, 8))
        
        # Filtrer les données cumulatives jusqu'à l'année courante
        cumulative_data = activity_trend[activity_trend['year'] <= year]
        
        # Tracer les lignes pour chaque groupe de codes postaux
        sns.lineplot(data=cumulative_data, x='year', y='count', hue='zipcode_group', marker='o', palette='tab10')
        
        plt.title(f"Tendances des activités par code postal jusqu'à {year}", fontsize=16)
        plt.xlabel("Année", fontsize=14)
        plt.ylabel("Nombre d'occurrences d'activités", fontsize=14)
        plt.legend(title="Code Postal", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        # Enregistrer l'image pour cette année
        frame_filename = os.path.join(frames_dir, f"frame_{year}.png")
        plt.savefig(frame_filename)
        plt.close()

    print(f"Frames sauvegardées dans le dossier '{frames_dir}'.")

# Créer un GIF à partir des frames enregistrées
def create_gif():
    print("Création du GIF...")
    images = []
    for file_name in sorted(os.listdir(frames_dir)):
        if file_name.endswith(".png"):
            file_path = os.path.join(frames_dir, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(gif_filename, images, fps=2)
    print(f"GIF créé avec succès: {gif_filename}")

# Supprimer les frames temporaires
def cleanup_frames():
    print("Nettoyage des frames temporaires...")
    for file_name in os.listdir(frames_dir):
        file_path = os.path.join(frames_dir, file_name)
        os.remove(file_path)
    os.rmdir(frames_dir)
    print("Frames temporaires supprimées.")

def main():
    print("Début du processus d'analyse des tendances des activités par code postal...")

    # Étape 1 : Récupérer les données de MongoDB
    data = fetch_data()

    # Étape 2 : Préparer les données
    df_prepared = prepare_data(data)

    # Étape 3 : Analyser les tendances des activités par code postal
    activity_trend = analyze_activity_trends_by_postal(df_prepared)

    # Étape 4 : Sauvegarder les frames par année
    save_frames_by_year(activity_trend)

    # Étape 5 : Créer le GIF
    create_gif()

    # Étape 6 : Nettoyer les frames temporaires
    cleanup_frames()

    print("Processus terminé.")

if __name__ == "__main__":
    main()


Début du processus d'analyse des tendances des activités par code postal...
Connexion à MongoDB...
Extraction des données de MongoDB...
Nombre d'entrées récupérées: 1890258
Préparation des données pour l'analyse...
Nombre d'entrées après préparation: 969879
Analyse des tendances des activités par code postal...
Création des frames pour chaque année...
Frames sauvegardées dans le dossier 'frames_postal'.
Création du GIF...


C:\Users\atdf2\AppData\Local\Temp\ipykernel_22060\1020227339.py:123: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


GIF créé avec succès: activity_trends_by_postal_code.gif
Nettoyage des frames temporaires...
Frames temporaires supprimées.
Processus terminé.
